In [ ]:
# third party
from recordlinkage.datasets import load_febrl4

# syft absolute
import syft as sy

# Create Nodes

## Staging Low side

create enclave node

In [ ]:
embassador_node_low = sy.Orchestra.launch(
    name="ambassador node",
    node_side_type="low",
    local_db=True,
    reset=True,
    #     enable_warnings=True,
)

Create canada node & italy node

In [ ]:
ca_node_low = sy.Orchestra.launch(
    name="canada-1",
    node_side_type="low",
    local_db=True,
    reset=True,
    #     enable_warnings=True,
)
it_node_low = sy.Orchestra.launch(
    name="italy-1",
    node_side_type="low",
    local_db=True,
    reset=True,
    #     enable_warnings=True,
)

In [ ]:
gateway_node_low = sy.orchestra.launch(
    name="gateway-1",
    node_type="gateway",
    node_side_type="low",
    local_db=True,
    reset=True,
    dev_mode=True,
    #                                        enable_warnings=True
)

## High side

In [ ]:
enclave_node_high = sy.orchestra.launch(
    name="enclave node",
    node_type="enclave",
    reset=True,
    #     enable_warnings=True,
)
ca_node_high = sy.Orchestra.launch(
    name="canada-2",
    local_db=True,
    reset=True,
    #     enable_warnings=True,
)
it_node_high = sy.Orchestra.launch(
    name="italy-2",
    local_db=True,
    reset=True,
    #     enable_warnings=True,
)

In [ ]:
gateway_node_high = sy.orchestra.launch(
    name="gateway-2",
    node_type="gateway",
    local_db=True,
    reset=True,
    dev_mode=True,
    #     enable_warnings=True,
)

# DOs

## Login

### Staging Low side

In [ ]:
do_ca_client_low = ca_node_low.login(email="info@openmined.org", password="changethis")
do_it_client_low = it_node_low.login(email="info@openmined.org", password="changethis")
embassador_client_low = embassador_node_low.login(
    email="info@openmined.org", password="changethis"
)

### Production High side

In [ ]:
do_ca_client_high = ca_node_high.login(
    email="info@openmined.org", password="changethis"
)
do_it_client_high = it_node_high.login(
    email="info@openmined.org", password="changethis"
)

## Connect to network

In [ ]:
# TODO: add security layer here

In [ ]:
enclave_client_high = enclave_node_high.client

In [ ]:
# gateway_root_client.register(name="", email="info@openmined.org", password="changethis")
# gateway_root_client.register(name="", email="info@openmined.org", password="changethis")

In [ ]:
res = do_ca_client_low.connect_to_gateway(
    handle=gateway_node_low
)  # add credentials here

In [ ]:
res = do_ca_client_low.connect_to_gateway(
    handle=gateway_node_low
)  # add credentials here
res = do_it_client_low.connect_to_gateway(
    handle=gateway_node_low
)  # add credentials here

In [ ]:
res = do_ca_client_high.connect_to_gateway(handle=gateway_node_high)
res = do_it_client_high.connect_to_gateway(handle=gateway_node_high)

In [ ]:
## Also for ambassador

In [ ]:
# TODO: who is going to be responsible for connecting the enclave to the gateway
res = enclave_client_high.connect_to_gateway(handle=gateway_node_high)

In [ ]:
res = embassador_client_low.connect_to_gateway(
    handle=gateway_node_low
)  # add credentials here

In [ ]:
## Upload dataset

### Staging Low side

In [ ]:
# Using public datasets from  Freely Extensible Biomedical Record Linkage (Febrl)
canada_census_data_low, italy_census_data_low = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data_low, italy_census_data_low],
    [do_ca_client_low, do_it_client_low],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

### Production High side

In [ ]:
# Using public datasets from  Freely Extensible Biomedical Record Linkage (Febrl)
canada_census_data_high, italy_census_data_high = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data_high, italy_census_data_high],
    [do_ca_client_high, do_it_client_high],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

## create accounts for DS

### Staging Low side

In [ ]:
for client in [do_ca_client_low, do_it_client_low]:
    client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )

In [ ]:
embassador_client_low.register(
    name="Sheldon",
    email="sheldon@caltech.edu",
    password="changethis",
    password_verify="changethis",
)

## Create account for embassador

### Production High Side

In [ ]:
for client in [do_ca_client_high, do_it_client_high]:
    client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )

# DS Low Side

## DS Get proxy clients

### Staging Low side

In [ ]:
ds_gateway_client_low = gateway_node_low.client

In [ ]:
assert len(ds_gateway_client_low.domains) == 3
ds_gateway_client_low.domains

In [ ]:
ds_ca_proxy_client_low = ds_gateway_client_low.domains[1].login(
    email="sheldon@caltech.edu", password="changethis"
)
ds_it_proxy_client_low = ds_gateway_client_low.domains[2].login(
    email="sheldon@caltech.edu", password="changethis"
)
ds_amb_proxy_client_low = ds_gateway_client_low.domains[0].login(
    email="sheldon@caltech.edu", password="changethis"
)

## Find datasets

In [ ]:
canada_census_data = ds_ca_proxy_client_low.datasets[-1].assets[0]
italy_census_data = ds_it_proxy_client_low.datasets[-1].assets[0]

## Create Request

In [ ]:
@sy.syft_function_single_use(
    canada_census_data=canada_census_data, italy_census_data=italy_census_data
)
def compute_census_matches(canada_census_data, italy_census_data):
    # third party
    import recordlinkage

    # Index step
    indexer = recordlinkage.Index()
    indexer.block("given_name")

    candidate_links = indexer.index(canada_census_data, italy_census_data)

    # Comparison step
    compare_cl = recordlinkage.Compare()

    compare_cl.exact("given_name", "given_name", label="given_name")
    compare_cl.string(
        "surname", "surname", method="jarowinkler", threshold=0.85, label="surname"
    )
    compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
    compare_cl.exact("suburb", "suburb", label="suburb")
    compare_cl.exact("state", "state", label="state")
    compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

    features = compare_cl.compute(
        candidate_links, canada_census_data, italy_census_data
    )

    # Classification step
    matches = features[features.sum(axis=1) > 3]

    return len(matches)

In [ ]:
# Checking result of mock data execution
mock_result = compute_census_matches(
    canada_census_data=canada_census_data.mock,
    italy_census_data=italy_census_data.mock,
)
mock_result

In [ ]:
ds_amb_proxy_client_low.code.request_code_execution(compute_census_matches)

# Ambassador flow

## Check Code Staging Low Side

In [ ]:
embassador_client_low.requests[0].code

## Login to Production High Side

In [ ]:
amb_gateway_client_high = gateway_node_high.client

In [ ]:
assert len(amb_gateway_client_high.domains) == 2
amb_gateway_client_high.domains

In [ ]:
amb_ca_proxy_client_high = amb_gateway_client_high.domains[1].login(
    email="sheldon@caltech.edu", password="changethis"
)
amb_it_proxy_client_high = amb_gateway_client_high.domains[0].login(
    email="sheldon@caltech.edu", password="changethis"
)

In [ ]:
assert len(amb_gateway_client_high.enclaves) == 1

In [ ]:
amb_ca_proxy_client_high

In [ ]:
amb_enclave_proxy_client_high = amb_gateway_client_high.enclaves[0].login(
    name="Sheldon", email="sheldon@caltech.edu", password="changethis", register=True
)

In [ ]:
# # this also creates a guest client
# embassador_client_high = enclave_node_high.login(email="info@openmined.org", password="changethis",
#                                                  name="Signor Ambassador", register=True)

## Find Datasets Production High side

In [ ]:
canada_census_data_high = amb_ca_proxy_client_high.datasets[-1].assets[0]
italy_census_data_high = amb_it_proxy_client_high.datasets[-1].assets[0]

Copy code from the request

## Submit code Production High side

In [ ]:
@sy.syft_function_single_use(
    canada_census_data=canada_census_data_high, italy_census_data=italy_census_data_high
)
def compute_census_matches_high(canada_census_data, italy_census_data):
    # third party
    import recordlinkage

    # Index step
    indexer = recordlinkage.Index()
    indexer.block("given_name")

    candidate_links = indexer.index(canada_census_data, italy_census_data)

    # Comparison step
    compare_cl = recordlinkage.Compare()

    compare_cl.exact("given_name", "given_name", label="given_name")
    compare_cl.string(
        "surname", "surname", method="jarowinkler", threshold=0.85, label="surname"
    )
    compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
    compare_cl.exact("suburb", "suburb", label="suburb")
    compare_cl.exact("state", "state", label="state")
    compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

    features = compare_cl.compute(
        candidate_links, canada_census_data, italy_census_data
    )

    # Classification step
    matches = features[features.sum(axis=1) > 3]

    return len(matches)

In [ ]:
# Checking result of mock data execution
mock_result = compute_census_matches_high(
    canada_census_data=canada_census_data_high.mock,
    italy_census_data=italy_census_data_high.mock,
)
mock_result

In [ ]:
# note that this is not embassador_client_high.**code**.request_code_execution
amb_enclave_proxy_client_high.request_code_execution(compute_census_matches_high)

## DOs Approve Production High Side

In [ ]:
do_ca_client_high.requests[0].approve()

In [ ]:
do_it_client_high.requests[0].approve()

## Embassdor gets result from Production High Side

In [ ]:
amb_enclave_proxy_client_high.code[-1].status

In [ ]:
result_pointer = amb_enclave_proxy_client_high.code.compute_census_matches_high(
    canada_census_data=canada_census_data_high,
    italy_census_data=italy_census_data_high,
)

result_pointer

In [ ]:
real_result = result_pointer.get()
real_result

## Ambassador Deposits Result

In [ ]:
embassador_client_low.requests[0].accept_by_depositing_result(real_result)

# DS

##  Get result from Staging Low Side

In [ ]:
ds_amb_proxy_client_low.code[-1].status

In [ ]:
result_pointer = ds_amb_proxy_client_low.code.compute_census_matches(
    canada_census_data=canada_census_data,
    italy_census_data=italy_census_data,
)

In [ ]:
result_pointer

In [ ]:
real_result = result_pointer.get()
real_result